# 4장. 마켓 타이밍 전략

In [1]:
# coding: utf-8
from typing import Dict, Optional

import pandas as pd
import plotly.express as px

from data.data_loader import PykrxDataLoader
from simulation.account import Account
from simulation.broker import Broker
from simulation.utility import get_lookback_fromdate, rebalance
from simulation.metric import cagr, mdd, sharpe_ratio, sortino_ratio

In [2]:
# 데이터 시작과 끝 날자 정의
fromdate = '2018-07-10'
todate = '2023-09-27'

# 투자할 종목 후보 정의
ticker_list = ['005930', '000660', '207940',
               '051910', '006400', '005380',
               '000270', '005490', '035420']

# 이동 평균 기간 정의
period = 3

# 기간을 고려한 데이터 시작 날짜 가져오기
adj_fromdate = get_lookback_fromdate(fromdate=fromdate, lookback=period, freq='m')

# 데이터 불러오기
data_loader = PykrxDataLoader(fromdate=adj_fromdate, todate=todate, market='KOSPI')
olcv_data = data_loader.load_stock_data(ticker_list=ticker_list, freq='m', delay=1)

# 데이터 확인하기
olcv_data.head()

/opt/anaconda3/envs/pyra312/lib/python3.12/site-packages/pykrx/stock/stock_api.py:50: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.resample("M").apply(how)
/Users/kms/dev/PyRA/data/data_loader.py:116: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.groupby('ticker').resample(freq).apply(rule).reset_index(level=0)


,ticker,open,high,low,close,volume
date,,,,,,
2018-05-31,000270,31750,31900,31150,31150,1468344
2018-06-30,000270,31100,34000,30450,30850,18456227
2018-07-31,000270,30850,32850,29450,31700,17534882
2018-08-31,000270,31650,32600,30950,32050,13377137
2018-09-30,000270,31900,35600,31350,35100,19573161


In [5]:
def get_moving_average_weights(ohlcv_data: pd.DataFrame, ma_data: pd.DataFrame) -> Optional[Dict]:
    # (1) 이동 평균 데이터 중 결측치가 있는지 확인함
    if ma_data.isnull().values.any():
        return None

    # (2) 매수할 주식과 매도할 주식을 선정함
    weights = {}
    stocks_to_buy = []
    for ticker in ohlcv_data['ticker']:
        # 종가 > 이동 평균
        if ohlcv_data.loc[ohlcv_data['ticker'] == ticker, 'close'].values > ma_data[ticker]:
            stocks_to_buy.append(ticker)
        # 종가 <= 이동 평균
        else:
            weights[ticker] = 0.0

    # 매수할 주식이 없는 경우 포트폴리오 반환
    if not stocks_to_buy:
        return weights

    # (3) 매수할 주식 비율을 할당함
    ratio = 1 / len(stocks_to_buy)
    for ticker in stocks_to_buy:
        weights[ticker] = ratio

    return weights

In [ ]:
def simulate_moving_average(ohlcv_data: pd.DataFrame,
                            ma_type: str,
                            period: int) -> Account:


In [ ]:
# 이동 평균 종류 정의
ma_type = 'sma' # {ema, sma}

# 단순 이동 평균 전략 실행하기
sma_simulation_account = simulate_moving_average()